In [21]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']='6ce270de-788d-4e07-8bae-3206860a6387'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='cvankaya_0000027070'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_config/bastion-key-cvankaya'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

In [22]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

In [23]:
SLICE_NAME = 'p4-lb-2'

# We will use Ubuntu 20.04 for both nodes
IMAGE = 'default_ubuntu_20'
CORES = 1
RAM = 2
DISK_R = 50
DISK_H = 20
SITE="UCSD"

routers = {"r1":{}, "r2":{}}
lb = "lb"
clients = {"c1":{}, "c2":{}}
servers = {"s1":{}, "s2":{}}

In [9]:
import datetime
from datetime import timedelta

#Create Slice
slice = fablib.new_slice(name=SLICE_NAME)
now = datetime.datetime.now(datetime.timezone.utc)
end_date = (now + timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S %z")
#slice.renew(end_date)

#Create Nodes
#LB Node
lb_node = slice.add_node(name="lb", site=SITE, cores=CORES, ram=RAM, image=IMAGE, disk=DISK_R)

#Server Nodes
for s in servers.keys():
    servers[s]["node"] = slice.add_node(name = s, site=SITE, cores=CORES, ram=RAM, image=IMAGE, disk=DISK_H)

#Router Nodes
for r in routers.keys():
    routers[r]["node"] = slice.add_node(name = r, site=SITE, cores=CORES, ram=RAM, image=IMAGE, disk=DISK_R)

#Client Nodes
for c in clients.keys():
    clients[c]["node"] = slice.add_node(name = c, site=SITE, cores=CORES, ram=RAM, image=IMAGE, disk=DISK_H)

#Create Network
#LB-Servers
for s, v in servers.items():
    lb_iface = lb_node.add_component(model='NIC_Basic', name="if_{}".format(s)).get_interfaces()[0]
    s_iface = v["node"].add_component(model='NIC_Basic', name="if").get_interfaces()[0]
    net = slice.add_l2network(name="net_lb_{}".format(s), interfaces=[lb_iface, s_iface])

#LB-Routers
for r, v in routers.items():
    lb_iface = lb_node.add_component(model='NIC_Basic', name="if_{}".format(r)).get_interfaces()[0]
    r_iface = v["node"].add_component(model='NIC_Basic', name="if_lb").get_interfaces()[0]
    net = slice.add_l2network(name="net_lb_{}".format(r), interfaces=[lb_iface, r_iface])

#Todo Router-Router

#Routers-Clients
for r, c in zip(routers.keys(), clients.keys()):
    r_iface = routers[r]["node"].add_component(model='NIC_Basic', name="if_c").get_interfaces()[0]
    c_iface = clients[c]["node"].add_component(model='NIC_Basic', name="if").get_interfaces()[0]
    net = slice.add_l2network(name="net_{}_{}".format(r,c), interfaces=[r_iface, c_iface])

In [10]:
#Submit Slice Request
slice.submit()


-----------  ------------------------------------
Slice Name   p4-lb-2
Slice ID     42aa5d35-08de-4391-a8a8-37e3d2247438
Slice State  StableOK
Lease End    2022-11-30 09:49:57 +0000
-----------  ------------------------------------

Retry: 12, Time: 239 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
a9ef9188-2209-446d-b268-d169ff43f08c  lb      UCSD    ucsd-w2.fabric-testbed.net        1      4     100  default_ubuntu_20  132.249.252.161  Active
17c5ec12-81a6-40dd-9e3e-e82d8ea91a0c  s1      UCSD    ucsd-w2.fabric-testbed.net        1      4     100  default_ubuntu_20  132.249.252.142  Active
c8a85ecd-e7ec-4cea-963e-75a3c0792a24  s2      UCSD    ucsd-w2.fabric-testbed.net        1      4     100  default_ubuntu_20  1

'42aa5d35-08de-4391-a8a8-37e3d2247438'

In [11]:
now = datetime.datetime.now(datetime.timezone.utc)
end_date = (now + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

In [12]:
#Create Subnets
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet1 = IPv4Network("192.168.1.0/24")
    ip_c1 = IPv4Address("192.168.1.1")
    
    subnet2 = IPv4Network("192.168.2.0/24")
    ip_c2 = IPv4Address("192.168.2.1")
    
    subnet3 = IPv4Network("192.168.3.0/24")
    ip_s1 = IPv4Address("192.168.3.1")
    ip_s2 = IPv4Address("192.168.3.2")
except Exception as e:
    print(f"Exception: {e}")

# Configure clients and servers

In [13]:
node = slice.get_node("s1")
iface = node.get_interface(network_name="net_lb_s1") 
iface.ip_addr_add(addr=ip_s1, subnet=subnet3)
print("Configuring IP {} for {}".format(ip_s1, "s1"))
stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
print (stdout)

node = slice.get_node("s2")
iface = node.get_interface(network_name="net_lb_s2") 
iface.ip_addr_add(addr=ip_s2, subnet=subnet3)
print("Configuring IP {} for {}".format(ip_s2, "s2"))
stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
print (stdout)

node = slice.get_node("c1")
iface = node.get_interface(network_name="net_r1_c1") 
iface.ip_addr_add(addr=ip_c1, subnet=subnet1)
print("Configuring IP {} for {}".format(ip_c1, "c1"))
stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
print (stdout)

node = slice.get_node("c2")
iface = node.get_interface(network_name="net_r2_c2") 
iface.ip_addr_add(addr=ip_c2, subnet=subnet2)
print("Configuring IP {} for {}".format(ip_c2, "c2"))
stdout, stderr = node.execute(f'ip addr show {iface.get_os_interface()}')
print (stdout)

Configuring IP 192.168.3.1 for s1
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 56:21:f2:62:91:ea brd ff:ff:ff:ff:ff:ff
    inet 192.168.3.1/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::5421:f2ff:fe62:91ea/64 scope link 
       valid_lft forever preferred_lft forever

Configuring IP 192.168.3.2 for s2
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 56:74:b4:68:66:dc brd ff:ff:ff:ff:ff:ff
    inet 192.168.3.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::5474:b4ff:fe68:66dc/64 scope link 
       valid_lft forever preferred_lft forever

Configuring IP 192.168.1.1 for c1
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 2e:a1:b4:ff:ee:4c brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global ens7
       valid_lft forever preferre

In [14]:
try:
    slice = fablib.get_slice(name=SLICE_NAME)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ---------------------------------------------------------------------------------------------------------------------------
ID                 a9ef9188-2209-446d-b268-d169ff43f08c
Name               lb
Cores              1
RAM                4
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               ucsd-w2.fabric-testbed.net
Site               UCSD
Management IP      132.249.252.161
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -J cvankaya_0000027070@bastion-1.fabric-testbed.net ubuntu@132.249.252.161
-----------------  ---------------------------------------------------------------------------------------------------------------------------
-----------------  ---------------------------------------------------------------------------------------------------------------------------
ID                 17c5ec12-81a6-40dd-9e3e-e82d8ea91a0c
Name               s1


In [15]:
node = slice.get_node("lb")
node.upload_file('./p4_setup.sh', '/home/ubuntu/p4_setup.sh')

<SFTPAttributes: [ size=794 uid=1000 gid=1000 mode=0o100664 atime=1669716346 mtime=1669716346 ]>

In [16]:
node = slice.get_node("r1")
node.upload_file('./p4_setup.sh', '/home/ubuntu/p4_setup.sh')

<SFTPAttributes: [ size=794 uid=1000 gid=1000 mode=0o100664 atime=1669716349 mtime=1669716350 ]>

In [17]:
node = slice.get_node("r2")
node.upload_file('./p4_setup.sh', '/home/ubuntu/p4_setup.sh')

<SFTPAttributes: [ size=794 uid=1000 gid=1000 mode=0o100664 atime=1669716352 mtime=1669716353 ]>

In [18]:
node = slice.get_node("lb")
node.upload_file('./simple_router.p4', '/home/ubuntu/simple_router.p4')

<SFTPAttributes: [ size=2727 uid=1000 gid=1000 mode=0o100664 atime=1669716355 mtime=1669716355 ]>

In [19]:
node = slice.get_node("r1")
node.upload_file('./simple_router.p4', '/home/ubuntu/simple_router.p4')

<SFTPAttributes: [ size=2727 uid=1000 gid=1000 mode=0o100664 atime=1669716358 mtime=1669716358 ]>

In [20]:
node = slice.get_node("r2")
node.upload_file('./simple_router.p4', '/home/ubuntu/simple_router.p4')

<SFTPAttributes: [ size=2727 uid=1000 gid=1000 mode=0o100664 atime=1669716361 mtime=1669716361 ]>

In [21]:
# Add routes

In [5]:
try:
    slice = fablib.get_slice(name=SLICE_NAME)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()